In [0]:
# 07_Time_Travel

# Set context
spark.sql("USE CATALOG main")
spark.sql("USE SCHEMA ecommerce")

# We will use 'gold_sales_by_state' for this demo because it is small (27 rows).
target_table = "gold_sales_by_state"

print(f"--- 1. CURRENT STATE of {target_table} ---")
# Count rows before the disaster
initial_count = spark.read.table(target_table).count()
print(f"Initial Row Count: {initial_count}")


# ====================================================
# SIMULATE ACCIDENTAL DELETE
# ====================================================
print("\n--- 2. SIMULATING ACCIDENTAL DELETE ---")
# "Oops! I deleted all records for SP (Sao Paulo) state!"
spark.sql(f"DELETE FROM {target_table} WHERE customer_state = 'SP'")

# Verify the damage
after_delete_count = spark.read.table(target_table).count()
print(f"Row Count after DELETE: {after_delete_count}")
print(f"Records lost: {initial_count - after_delete_count}")


# ====================================================
# CHECK HISTORY
# ====================================================
print("\n--- 3. CHECKING TABLE HISTORY ---")
df_history = spark.sql(f"DESCRIBE HISTORY {target_table}")
display(df_history.select("version", "timestamp", "operation", "operationMetrics.numOutputRows"))

# We want to go back to the version BEFORE the delete.
# The DELETE operation will be the latest version. We need (Latest Version - 1).
last_version = df_history.first()["version"]
restore_version = last_version - 1

print(f"Current Version: {last_version} (This is the DELETE)")
print(f"Restoring to Version: {restore_version} (This is the clean state)")


# ====================================================
# RESTORE (TIME TRAVEL)
# ====================================================
print(f"\n--- 4. RESTORING TABLE TO VERSION {restore_version} ---")
spark.sql(f"RESTORE TABLE {target_table} TO VERSION AS OF {restore_version}")

print("--> Restore Command Executed.")


# ====================================================
# VALIDATE ROLLBACK
# ====================================================
print("\n--- 5. VALIDATING ROLLBACK ---")
final_count = spark.read.table(target_table).count()
print(f"Final Row Count: {final_count}")

if final_count == initial_count:
    print("SUCCESS: Data successfully restored! Magic confirmed.")
else:
    print("ERROR: Row count does not match initial state.")